In [1]:
import tensorflow as tf

In [2]:
import numpy as np
fea = np.random.normal(size=(6,2)).astype(np.float32)

In [3]:
tf.InteractiveSession()

In [4]:
embedded_feature = tf.constant(value=fea)
pids=tf.constant(value=[0,0,0,0,0,1])
pids.eval().shape

(6,)

In [5]:
cdist = tf.expand_dims(embedded_feature,axis=1) - tf.expand_dims(embedded_feature,axis=0) 
cdist =tf.sqrt(tf.reduce_sum(tf.square(cdist), axis=-1) + 1e-12)
cdist.eval().shape

(6, 6)

In [6]:
same_identity_mask = tf.equal(tf.expand_dims(pids, axis=1), tf.expand_dims(pids, axis=0))
same_identity_mask.eval()

array([[ True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True, False],
       [False, False, False, False, False,  True]], dtype=bool)

In [7]:
negative_mask = tf.logical_not(same_identity_mask)
negative_mask.eval()

array([[False, False, False, False, False,  True],
       [False, False, False, False, False,  True],
       [False, False, False, False, False,  True],
       [False, False, False, False, False,  True],
       [False, False, False, False, False,  True],
       [ True,  True,  True,  True,  True, False]], dtype=bool)

In [8]:
positive_mask = tf.logical_xor(same_identity_mask,tf.eye(tf.shape(pids)[0], dtype=tf.bool))
positive_mask.eval()

array([[False,  True,  True,  True,  True, False],
       [ True, False,  True,  True,  True, False],
       [ True,  True, False,  True,  True, False],
       [ True,  True,  True, False,  True, False],
       [ True,  True,  True,  True, False, False],
       [False, False, False, False, False, False]], dtype=bool)

In [9]:
furthest_positive = tf.reduce_max(cdist*tf.cast(positive_mask, tf.float32), axis=1)
furthest_positive.eval()

array([ 1.75529301,  1.96527934,  2.06328487,  2.06328487,  1.92065263,  0.        ], dtype=float32)

In [18]:
margin = 0.5
def computeDistanceOfPosNegPair(x):
    x[0].set_shape([None])
    x[1].set_shape([None])
    x[2].set_shape([None])
    pos_pair_dist = tf.expand_dims(tf.boolean_mask(x[0], x[1]),axis=1)
    neg_pair_dist = tf.expand_dims(tf.boolean_mask(x[0], x[2]),axis=0)
            
    #check if there is no positive pairs in the batch
    noPosPairs = tf.less(tf.shape(pos_pair_dist)[0],1)
    #check if there is no negative pairs in the batch
    noNegPairs = tf.less(tf.shape(neg_pair_dist)[1],1)
            
    def PosNegPairs(): return tf.reshape(pos_pair_dist - neg_pair_dist,shape=[-1])
    def noPosPair(): return  tf.reshape(-neg_pair_dist,shape=[-1])
    def noNegPair(): return tf.reshape(pos_pair_dist,shape=[-1])
    pos_neg_dist = tf.case({noPosPairs:noPosPair, \
                            noNegPairs:noNegPair}, \
                            default=PosNegPairs,exclusive=True)
    hardcases,_ = tf.nn.top_k(-pos_neg_dist,k=3)
    hardcases = -hardcases
    hardcases = tf.maximum(hardcases+margin,0.0)
    
    return hardcases
            
    #weighted_pos_neg_dist = tf.multiply(tf.nn.softmax(pos_neg_dist),pos_neg_dist)
    #return tf.reduce_sum(weighted_pos_neg_dist)

pos_neg_dist = tf.map_fn(computeDistanceOfPosNegPair,(cdist, positive_mask, negative_mask), tf.float32)
pos_neg_dist.eval()

array([[ 0.        ,  0.08740103,  1.04532599],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.59311676],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.01517606],
       [ 0.        ,  0.        ,  0.        ]], dtype=float32)

In [11]:
#diff = tf.nn.softplus(pos_neg_dist)
diff = tf.reduce_mean(tf.maximum(pos_neg_dist + 0.5, 0.0)) +1e-12
diff.eval()

0.66666669

In [58]:
def reduceMin(x):
    x[0].set_shape([None])
    x[1].set_shape([None])
    return tf.reduce_min(tf.boolean_mask(x[0], x[1]))

closest_negative = tf.map_fn(reduceMin,(cdist, negative_mask), tf.float32)
closest_negative.eval()

array([ inf,  inf,  inf,  inf,  inf,  inf,  inf], dtype=float32)

In [14]:
closest_negative = cdist*tf.cast(negative_mask,tf.float32)
closest_negative.eval()

array([[ 0.        ,  1.10811687,  0.86912888,  1.5635612 ,  1.47273374,
         1.49304783,  0.45302168],
       [ 1.10811687,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.86912888,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 1.5635612 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 1.47273374,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 1.49304783,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.45302168,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [15]:
closest_negative = tf.reduce_min(cdist + 1e5*tf.cast(same_identity_mask, tf.float32), axis=1)
closest_negative.eval()

array([ 0.45302168,  1.10811687,  0.86912888,  1.5635612 ,  1.47273374,
        1.49304783,  0.45302168], dtype=float32)

In [ ]:
tf.nn.top_k